# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Check the current working directory
print(os.getcwd())

/home/workspace


In [3]:
# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
filepath

'/home/workspace/event_data'

In [4]:
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extract each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
# print(full_data_rows_list)

# Create a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If there is a locally installed Apache Cassandra instance
    session = cluster.connect() # To establish connection and begin executing queries, need a session

except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### With Apache Cassandra it is best practice to model the database tables on the queries that are of interest.

#### Queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1

In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Table Name: songs_played
# column 1: sessionID
# column 2: itemInSession
# column 3: artist
# column 4: song
# column 5: length
# PRIMARY KEY(sessionID, itemInSession)

query = "CREATE TABLE IF NOT EXISTS songs_played "
query = query + """(sessionID int, 
                    itemInSession int, 
                    artist text, song text, 
                    length float, 
                    PRIMARY KEY (sessionID, itemInSession))""" 
                    # the combination of sessionID and itemInSession makes each row unique
try:
    session.execute(query)
except Exception as e:
    print(e)

##### For Query 1 the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

In [11]:
## Query 1:  Set up the CSV file and insert into Apache Cassandra table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        # print(line)
        # print("sessionID: " + line[8] + ", itemInSession: " + line[3] +", artist: " + line[0] + ", song: " + line[9] + ", length: " + line[5])
        
        query = "INSERT INTO songs_played (sessionID, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


In [12]:
## Query 1:  SELECT statement to verify the data was entered into the table

query_artist_title_length= ("""SELECT artist, song, length
                                    FROM songs_played
                                    WHERE sessionID = 338 
                                    AND itemInSession = 4 """)

try:
    rows = session.execute(query_artist_title_length)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length,)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2

In [14]:
## Query 2:   Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Table Name: user_songs
# column 1: userID
# column 2: sessionID
# column 3: artist
# column 4: song
# column 5: user_first_name
# column 6: user_last_name
# column 7: itemInSession
# PRIMARY KEY(userID, sessionID)

query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + """(userID int, 
                  sessionID int,
                  artist text, 
                  song text, 
                  user_first_name text, 
                  user_last_name text, 
                  itemInSession int, 
                  PRIMARY KEY ((userID, sessionID), itemInSession))"""
                  # to geht the data sorted by itemInSession it is necessary to make it a 
try:
    session.execute(query)
except Exception as e:
    print(e)

##### For Query 2 the Primary Key has three fields: userID and sessionID are the partition keys, and itemInSession is clustering key. Partitioning is done by userID and sessionID and within that partition, rows are ordered by the itemInSession. I decided against making both, sessionid and iteminsession, Clustering Columns to have the row sorting done on disk.

In [15]:
## Query 2:  Set up the CSV file and insert into Apache Cassandra table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        """print(line)
        print("userID: " + line[10] + 
              ", sessionID: " + line[8] +
              ", artist: " + line[0] + 
              ", song: " + line[9] + 
              ", user_first_name: " + line[1] +
              ", user_last_name: "+ line[4] +
              ", itemInSession: " + line[3])"""
        
        query = "INSERT INTO user_songs (userID, sessionID, artist, song, user_first_name, user_last_name, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

In [16]:
## Query 2:  SELECT statement to verify the data was entered into the table

query_artist_user_songs = ("""SELECT artist, song, user_first_name, user_last_name
                                    FROM user_songs
                                    WHERE userID = 10
                                    AND sessionID = 182""")

try:
    rows = session.execute(query_artist_user_songs)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.user_first_name, row.user_last_name,)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Table Name: song_users
# column 1: song
# column 2: userID
# column 3: user_first_name
# column 4: user_last_name
# PRIMARY KEY(song, userID)

query = "CREATE TABLE IF NOT EXISTS song_users "
query = query + """(song text, 
                    userID int,  
                    user_first_name text, 
                    user_last_name text, 
                    PRIMARY KEY (song, userID))"""

try:
    session.execute(query)
except Exception as e:
    print(e)        

##### For Query 3 the Primary Key has two fields: song is the partition key, and userID is clustering key. Partitioning is done by song and within that partition, rows are ordered by the userID.

In [18]:
## Query 3:  Set up the CSV file and insert into Apache Cassandra table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        """print(line)
        print(", song: " + line[9] + 
              ", userID: " + line[10] +             
              ", user_first_name: " + line[1] +
              ", user_last_name: "+ line[4])"""
        
        query = "INSERT INTO song_users (song, userID, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [19]:
## Query 3:  SELECT statement to verify the data was entered into the table

query_artist_song_users = ("""SELECT user_first_name, user_last_name
                                    FROM song_users
                                    WHERE song = 'Pump It'""")

try:
    rows = session.execute(query_artist_song_users)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_first_name, row.user_last_name,)

Sylvie Cruz
Kate Harrell
Jayden Fox


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the sessions

try:
    session.execute("DROP TABLE IF EXISTS songs_played")
    session.execute("DROP TABLE IF EXISTS user_songs")
    session.execute("DROP TABLE IF EXISTS song_users")
except Exception as error:
    print(error)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()